In [1]:
import json
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import os
import tripy
plt.style.use('classic')


## This file is used for I/O tests

In [2]:
from to3D import *
from toOBJ import *
loc="Bronew.json"
file=open(loc,"r")
data=json.loads(file.read())


In [3]:
typ=list()
ids=list()
for i in data["features"]:
    typ.append(i["type"])
    ids.append(i["id"])
print(typ)
print(ids)


['Feature', 'Polygon', 'GeometryCollection', 'GeometryCollection', 'GeometryCollection', 'GeometryCollection', 'MultiPolygon', 'MultiPolygon', 'MultiPolygon', 'MultiPolygon', 'MultiPolygon']
['values', 'earth', 'roads', 'walls', 'rivers', 'planks', 'buildings', 'prisms', 'squares', 'greens', 'fields']


In [4]:
earth=data["features"][1]["coordinates"]
roads=data["features"][2]["geometries"]
buildings=data["features"][6]["coordinates"]
fields=data["features"][10]["coordinates"]
square=data["features"][8]["coordinates"]
prisms=data["features"][7]["coordinates"]

In [5]:
#you can specify the width here, common for all roads though. modify getRoads for another behavior
earthtri,earthn = getEarth(earth[0])
roadtri,roadn = getRoads(roads)
fieldtri,fieldn = getFields(fields)
rooftri,roofn = getSurfaces(buildings,8)
wallbtri,wallbn = getWallsBuildings(buildings,8)
squaretri,squaren = getSurfaces(square,0.1)
prismstri,prismsn = getSurfaces(prisms,0.1)

In [6]:
def toStr(v):
    return str(v[0])+" "+str(v[1])+" "+str(v[2])
def getEarthColor():
    i=4
    kd=[0.26,0.53,0.19]
    ks=[0.2,0.5,0.2]
    ka=[0.4,0.4,0.4]
    tf=[1.0,1.0,1.0]
    ns=20
    ni=1.0
    return i,kd,ks,ka,tf,ns,ni

def getRoadColor():
    i=4
    kd=[0.46,0.53,0.49]
    ks=[0.2,0.2,0.2]
    ka=[0.4,0.4,0.4]
    tf=[1.0,1.0,1.0]
    ns=20
    ni=1.0
    return i,kd,ks,ka,tf,ns,ni

def getWallbColor():
    i=4
    kd=[0.46,0.03,0.09]
    ks=[0.3,0.01,0.02]
    ka=[0.2,0.01,0.01]
    tf=[1.0,1.0,1.0]
    ns=30
    ni=1.0
    return i,kd,ks,ka,tf,ns,ni
def getRoofColor():
    i=4
    kd=[0.33,0.35,0.38]
    ks=[0.23,0.25,0.28]
    ka=[1,1,1]
    tf=[1.0,1.0,1.0]
    ns=40
    ni=1.0
    return i,kd,ks,ka,tf,ns,ni
def getFieldColor():
    i=4
    kd=[0.95,0.80,0.65]
    ks=[0.93,0.85,0.68]
    ka=[1,0.8,0.6]
    tf=[1.0,1.0,1.0]
    ns=40
    ni=1.0
    return i,kd,ks,ka,tf,ns,ni
def getSquareColor():
    i=4
    kd=[0.45,0.30,0.25]
    ks=[0.93,0.85,0.68]
    ka=[1,0.8,0.6]
    tf=[1.0,1.0,1.0]
    ns=10
    ni=1.0
    return i,kd,ks,ka,tf,ns,ni
def getPrismsColor():
    i=4
    kd=[0.5,0.5,0.5]
    ks=[0.93,0.85,0.68]
    ka=[1,0.8,0.6]
    tf=[1.0,1.0,1.0]
    ns=40
    ni=1.0
    return i,kd,ks,ka,tf,ns,ni
def getMaterial(name):
    if name == "earth":
        return getEarthColor()
    if name == "road":
        return getRoadColor()
    if name == "field":
        return getFieldColor()
    if name == "wallb":
        return getWallbColor()
    if name == "roof":
        return getRoofColor()
    if name == "square":
        return getSquareColor()
    if name == "prisms":
        return getPrismsColor()
    
def writeMTL(objs,file,name):
    '''
    write the material file for all the objects in objs
    '''
    for i in range(len(objs)):
        il,kd,ks,ka,tf,ns,ni=getMaterial(name)
        file.write("newmtl "+name+"_"+str(i)+"\n")
        file.write("\t illum "+str(il)+"\n")
        file.write("\t Kd "+toStr(kd)+"\n")
        file.write("\t Ks "+toStr(ks)+"\n")
        file.write("\t Ka "+toStr(ka)+"\n")
        file.write("\t Tf "+toStr(tf)+"\n")
        file.write("\t Ns "+str(ns)+"\n")
        file.write("\t Ni "+str(ni)+"\n")

def writeObjects(objs,c,name):
    
    file.write("g "+name+"\n")
    i=0
    for o in objs:#there is one normal per triangle, so it's ok
        file.write("o "+name+"_"+str(i)+"\n")
        file.write("usemtl "+name+"_"+str(i)+"\n")
        writeFaces(len(o),c,file)
        i+=1
        c+=len(o)*3
        
    return c

def writeFaces(n,i,file):
    #i=currentCount, n lenght of this one
    #one line per triangle
    for id in range(n):
        file.write("f ")
        step=i+3*id
        file.write(str(step)+"//"+str(int((step+2)/3))+" ")
        
        file.write(str(step+1)+"//"+str(int((step+2)/3))+" ")
        
        file.write(str(step+2)+"//"+str(int((step+2)/3))+"\n")
        
    
def writeTriangleArray(array,file):
    '''
    writes in the file the array of triangles
    '''
    for a in array:
        for t in a:
            file.write(vertex(t,False))
        
def writeNormalArray(array,file):
    '''
    writes in the file the array of vertices
    '''
    for a in array:
        file.write(vertex(a,True))
        
def vertex(v,n=False):
    x,y,z=v
    s=""
    if(n):
        s="vn "
    else:
        s="v "

    s+=str(x)+" "+str(y)+" "+str(z)+"\n"
    return s

In [7]:
fn="filetest"
file=open(fn+".obj","w")
file.write("use mtllib "+fn+".mtl\n")

writeTriangleArray(earthtri,file)
for road in roadtri:
    writeTriangleArray(road,file)
for field in fieldtri:
    writeTriangleArray(field,file)
for roof in rooftri:
    writeTriangleArray(roof,file)
for wallb in wallbtri:
    writeTriangleArray(wallb,file)
for s in squaretri:
    writeTriangleArray(s,file)
for p in prismstri:
    writeTriangleArray(p,file)

writeNormalArray(earthn,file)
for road in roadn:
    writeNormalArray(road,file)
for field in fieldn:
    writeNormalArray(field,file)
for roof in roofn:
    writeNormalArray(roof,file)
for wallb in wallbn:
    writeNormalArray(wallb,file)
for s in squaren:
    writeNormalArray(s,file)
for p in prismsn:
    writeNormalArray(p,file)

counter=1
counter=writeObjects([earthtri],counter,"earth")
counter=writeObjects(roadn,counter,"road")
counter=writeObjects(fieldn,counter,"field")
counter=writeObjects(roofn,counter,"roof")
counter=writeObjects(wallbn,counter,"wallb")
counter=writeObjects(squaren,counter,"square")
writeObjects(prismsn,counter,"prisms")

mtl=open(fn+".mtl","w")

writeMTL(earthn,mtl,"earth")
writeMTL(roadn,mtl,"road")
writeMTL(fieldn,mtl,"field")
writeMTL(roofn,mtl,"roof")
writeMTL(wallbn,mtl,"wallb")
writeMTL(squaren,mtl,"square")
writeMTL(prismsn,mtl,"prisms")

In [8]:
prismsn

[array([[0, 0, 1],
        [0, 0, 1]])]

In [9]:
len(roadtri[0])

23

In [10]:
[1,1,1]+[2,3]

[1, 1, 1, 2, 3]